In [1]:
%load_ext autoreload
%autoreload 2

from pandas import read_excel
import csv
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from modAL.models import ActiveLearner
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem,DataStructs,Draw,PandasTools,Descriptors
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.Draw import IPythonConsole
from rdkit import RDLogger

from IPython.display import Image

# Configure the logging - RDKit is rather verbose..
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)
# Set the molecule representation to be SVG 
PandasTools.molRepresentation='svg'
# Loading and standardization method for SMILES -> RDKit molecule object
uncharger = rdMolStandardize.Uncharger()

import functions as fun

In [2]:
# Loading data sets
specs = pd.read_csv('/home/jovyan/covid-data/SPECS_ready_3.csv')
sss_batchA = pd.read_csv('/home/jovyan/covid-data/COVID_BatchA_ready_3.csv')
specs_version2= pd.read_csv("/home/jovyan/covid-data/specsSarsCov2-v1-CMPDannot.csv")

#Creating final dataset
data = pd.merge(specs[['ID_library', 'SMILES']], sss_batchA[['ID_covid', 'Intensity_MeanIntensity_illumMITO_cells.1','Labels']], 
                left_on= 'ID_library', right_on='ID_covid', how='left')
data = data.drop(['ID_covid'],axis=1)
data.head() 

,ID_library,SMILES,Intensity_MeanIntensity_illumMITO_cells.1,Labels
0,CBK309065,CC1=NN=C2N1C(SC(C)=C3C)=C3C(C4=CC=C(Cl)C=C4)=N...,0.009294,0
1,CBK041145,[H][C@@]12COP([O-])(=O)O[C@@]1([H])[C@@H](O)[C...,0.009393,0
2,CBK200894,CC1CS(=O)(=O)CCN1N=Cc1ccc(o1)[N+]([O-])=O,0.009726,0
3,CBK309015,O=C(N1)SC(CC2=CC=C(OCC(N3C)=NC4=C(C=CC=C4)C3=O...,0.011022,0
4,CBK041791,Fc1ccc(cc1)C(c2ccccc2)(c3ccccc3F)n4ccnc4,0.009869,0


In [3]:
# First we need to parse the SMILES and create RDKit 'molecule' objects 
# the 'standardize' function was the one we defined in the first cell with code in this file
data['MOL'] = data['SMILES'].apply(fun.standardize)
data.head()

,ID_library,SMILES,Intensity_MeanIntensity_illumMITO_cells.1,Labels,MOL
0,CBK309065,CC1=NN=C2N1C(SC(C)=C3C)=C3C(C4=CC=C(Cl)C=C4)=N...,0.009294,0,"<svg xmlns=""http://www.w3.org/2000/svg"" xmlns:..."
1,CBK041145,[H][C@@]12COP([O-])(=O)O[C@@]1([H])[C@@H](O)[C...,0.009393,0,"<svg xmlns=""http://www.w3.org/2000/svg"" xmlns:..."
2,CBK200894,CC1CS(=O)(=O)CCN1N=Cc1ccc(o1)[N+]([O-])=O,0.009726,0,"<svg xmlns=""http://www.w3.org/2000/svg"" xmlns:..."
3,CBK309015,O=C(N1)SC(CC2=CC=C(OCC(N3C)=NC4=C(C=CC=C4)C3=O...,0.011022,0,"<svg xmlns=""http://www.w3.org/2000/svg"" xmlns:..."
4,CBK041791,Fc1ccc(cc1)C(c2ccccc2)(c3ccccc3F)n4ccnc4,0.009869,0,"<svg xmlns=""http://www.w3.org/2000/svg"" xmlns:..."


### There is a problem with some of the smiles as they return none when trying to convert to molecule. The next step is to fix those cases which may be a product of a typo. 

In [4]:
corrupted_smiles_idx_list = data[data['MOL'].isnull()].index.tolist() #finding None values
data[data['MOL'].isnull()]

,ID_library,SMILES,Intensity_MeanIntensity_illumMITO_cells.1,Labels,MOL
131,CBK309029,O=S(OC1=CC([C@H](O)[C@@H](O)C(NC(C2N(C(C(NC([C...,0.009792,0,None
339,CBK309090,O=C(CC(N)CCCN)NCC(C(N/C(C(NC(C1NC(NCC1)=N)C2=O...,0.009078,0,None
418,CBK309764,CC1=C(CCC(CCCC(C)CCCC(C)CCCC(C)C)(C)O2)C2=C(C)...,0.011273,0,None
447,CBK278005,O=C(C1=C(N)C(C(C)=C2OC3=C(N=C21)C(C(N[C@@H]4C(...,0.009102,0,None
472,CBK309248,C[C@]([C@@](C[C@H]1O)([H])[C@]2(CC[C@@H]3O[C@@...,0.009041,0,None
512,CBK200645,CC[C@H](C)[C@H]1O[C@]2(CC[C@@H]1C)C[C@@H]1C[C@...,0.009255,0,None
682,CBK200530,C[C@H]1O[C@H](C[C@H](O)[C@@H]1O[C@H]1C[C@H](O)...,0.009501,0,None
720,CBK309019G,OCC1[N]2(CC[N]3(C4)CC[N]5(C6)CC[N]7(C8)CC2)[Gd...,0.009839,0,None
790,CBK309763G,O=C1[O-][Sb+3]([O-]C(C2[O-]3)C4=O)([O-]4)[O-]C...,0.009371,0,None
857,CBK309767G,O=C1[O-][Mn+2]([N]2(C3)CC4=C5COP(O)(O)=O)([O-]...,0.012623,0,None


In [5]:
specs_v2_smiles = specs_version2[['cbkid', 'smiles']] #Bringing second SPECS data set for comparision

for idx in corrupted_smiles_idx_list:
    compound_in_data = data['ID_library'].values[idx]
    idx_in_specsv2 = specs_v2_smiles.loc[specs_v2_smiles['cbkid'] == compound_in_data].index.tolist()[0]
    #compound_in_specsv2 = specs_v2_smiles['cbkid'].values[idx_in_specsv2]
    smiles_in_data = data['SMILES'].values[idx]
    smiles_in_specs_v2 = specs_v2_smiles['smiles'].values[idx_in_specsv2]
    try:
        mol = Chem.MolFromSmiles(smiles_in_specs_v2)
        print('Molecule successful')
        data['SMILES'] = data['SMILES'].replace(smiles_in_data, smiles_in_specs_v2)
        print('Replacement done')
    except 'None here':
        pass